1. Importing required libraries

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error


2.Custom Decision Tree implementation

In [9]:
class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def _information_gain(self, parent, left, right):
        wl = len(left) / len(parent)
        wr = len(right) / len(parent)
        return self._entropy(parent) - (wl * self._entropy(left) + wr * self._entropy(right))

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature in range(X.shape[1]):
            for threshold in np.unique(X[:, feature]):
                left = y[X[:, feature] <= threshold]
                right = y[X[:, feature] > threshold]
                if len(left) == 0 or len(right) == 0:
                    continue
                gain = self._information_gain(y, left, right)
                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature, threshold)

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        feature, threshold = best_split
        mask = X[:, feature] <= threshold

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[mask], y[mask], depth+1),
            'right': self._build_tree(X[~mask], y[~mask], depth+1)
        }

    def _predict_one(self, x, tree):
        if 'class' in tree:
            return tree['class']
        if x[tree['feature']] <= tree['threshold']:
            return self._predict_one(x, tree['left'])
        return self._predict_one(x, tree['right'])

    def predict(self, X):
        return [self._predict_one(x, self.tree) for x in X]

3.Iris dataset - Training and evaluation

In [10]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)
custom_preds = custom_tree.predict(X_test)

sk_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_tree.fit(X_train, y_train)
sk_preds = sk_tree.predict(X_test)

custom_acc = accuracy_score(y_test, custom_preds)
sk_acc = accuracy_score(y_test, sk_preds)

print("Decision Tree Accuracy Comparison (Iris Dataset):\n")
print(f"Custom Decision Tree Accuracy      : {custom_acc:.4f}")
print(f"Scikit-learn Decision Tree Accuracy: {sk_acc:.4f}")

if custom_acc > sk_acc:
    print("Observation: Custom tree performs better on unseen data.")
elif custom_acc < sk_acc:
    print("Observation: Scikit-learn tree performs better due to optimizations.")
else:
    print("Observation: Both models perform equally well.")


Decision Tree Accuracy Comparison (Iris Dataset):

Custom Decision Tree Accuracy      : 1.0000
Scikit-learn Decision Tree Accuracy: 1.0000
Observation: Both models perform equally well.


4. Ensemble Methods- classification (Wine dataset)

In [11]:
wine = load_wine()
X, y = wine.data, wine.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

dt_f1 = f1_score(y_test, dt.predict(X_test), average='weighted')
rf_f1 = f1_score(y_test, rf.predict(X_test), average='weighted')

print("F1 Score Comparison (Wine Dataset) :\n")
print(f"Decision Tree Classifier F1 Score : {dt_f1:.4f}")
print(f"Random Forest Classifier F1 Score : {rf_f1:.4f}")

print("\nConclusion:")
print("Random Forest generally achieves a higher F1 score because")
print("it combines multiple decision trees and reduces overfitting.")

F1 Score Comparison (Wine Dataset) :

Decision Tree Classifier F1 Score : 0.9440
Random Forest Classifier F1 Score : 1.0000

Conclusion:
Random Forest generally achieves a higher F1 score because
it combines multiple decision trees and reduces overfitting.


5. Hyperparameter Tuning - GridSearchCV

In [12]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    scoring='f1_weighted',
    cv=5
)

grid.fit(X_train, y_train)

print("GridSearchCV Results:\n")
print("Best Hyperparameters:")
for k, v in grid.best_params_.items():
    print(f" - {k}: {v}")

print(f"Best Cross-Validated F1 Score: {grid.best_score_:.4f}")
print("GridSearchCV evaluates multiple combinations using cross-validation.")


GridSearchCV Results:

Best Hyperparameters:
 - max_depth: None
 - min_samples_split: 2
 - n_estimators: 100
Best Cross-Validated F1 Score: 0.9783
GridSearchCV evaluates multiple combinations using cross-validation.


6. Regression models

In [13]:
X, y = wine.data, wine.target.astype(float)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt_reg = DecisionTreeRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)

dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

dt_mse = mean_squared_error(y_test, dt_reg.predict(X_test))
rf_mse = mean_squared_error(y_test, rf_reg.predict(X_test))

print("Regression Performance (MSE):\n")
print(f"Decision Tree Regressor MSE : {dt_mse:.4f}")
print(f"Random Forest Regressor MSE : {rf_mse:.4f}")
print("Lower MSE indicates better predictive performance.")

Regression Performance (MSE):

Decision Tree Regressor MSE : 0.1667
Random Forest Regressor MSE : 0.0648
Lower MSE indicates better predictive performance.


7. Hyperparameter Tuning - RandomizedSearchCV

In [14]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=5,
    cv=5,
    random_state=42
)

random_search.fit(X_train, y_train)

print("RandomizedSearchCV Results :\n")
print("Best Parameters Found:")
for k, v in random_search.best_params_.items():
    print(f" - {k}: {v}")

print(f"Best Cross-Validated Score: {random_search.best_score_:.4f}")
print("RandomizedSearchCV is efficient for large search spaces.")


RandomizedSearchCV Results :

Best Parameters Found:
 - n_estimators: 200
 - min_samples_split: 10
 - max_depth: None
Best Cross-Validated Score: 0.9211
RandomizedSearchCV is efficient for large search spaces.
